In [1]:
import glob
from math import ceil
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras import optimizers, losses
from keras.models import Model
import numpy as np
import cv2
import matplotlib.pyplot as plt

### The below method sends the label/file name of each image

In [2]:
def get_label(name):
    if name:
        img_name = name.split("/dataset/")
        img_name = img_name[1].split(".jpg")
        return img_name[0]
    return None

### Running the below cell stores labels of all the images in a list "labels"

In [5]:
img_path = './dataset/*.jpg'
images = glob.glob(img_path)
labels = []
k=0
for img in images:
    # get the image filename and set it as the label
    label_id = get_label(img)
    labels.append(label_id)
    k += 1

In [6]:
# Considering 80% of dataset and their labels for training 
length=int((len(labels)*80/100))
train_sample = images[0:length]
train_label= labels[0:length]

In [ ]:
#Given images are of 512*512 shape but reshaping it to 224*224 in the code
width = 224 
height = 224
n_color_channels = 3  # RGB color image
data_shape = (0, width, height, 3) # shape for images

In [10]:
# defines the batch size
batch_size = 64
# create batches of batch_size and make a list
batches_list = list(range(int(ceil(float(len(train_sample)) / batch_size))))
print('batches list: ', batches_list)

batches list:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]


### Autoencoder model:
In the below autoencoder model in the encoder part three 2D convolutional layers are used and each conv layer is followed by a max pooling layer with reLU activation function.
In the decoder part there are three upsampling layers used where each upsampling layer is followed by a 2D covolutional layer with reLU activation function and sigmoid activation function for the output layer of decoder

In [11]:
def autoencoder_model():
    # Encoder #
    input_img = Input(shape=(224, 224, 3)) # input image shape -> width = height = 224, no.of color channels = 3
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same', name='encoder')(x)

    # Decoder #
    x = UpSampling2D((2,2))(encoded)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    # create the autoencoder model #
    autoencoder = Model(input_img, decoded)
    return autoencoder

In [16]:
# Uses Stochastic gradient descent optimizer with learning rate 0.01
learning_rate=0.01
sgd = optimizers.SGD(lr=learning_rate, clipnorm=1.)
#Load the model
model = autoencoder_model()
# Loss = Mean Squared Error
model.compile(optimizer=sgd, loss=losses.mean_squared_error)

### Training Model:
In the below train_model function we load each batch of images (batch_size=64) and its labels. We create noise in the images with a noise factor of 0.5 and original images and the noisy images are sent for training with training batch_size of 16. The same procedure is used for each batch of images.
###### Note: 
Labels are just used for reference purpose, the labels are not used in the training

In [17]:
def train_model(model,batches_list,batch_size,train_sample,epochs):
    # Enumerate over the batch list with each batch of size 64
    for n, i in enumerate(batches_list):
        i_s = i * batch_size
        i_e = min([(i + 1) * batch_size, len(train_sample)])
        train_images=[]
        train_labels=[]
        for j in range(i_s,i_e):
            img_addr = train_sample[j]
            img = cv2.imread(img_addr)
            img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)
            img_lbl = get_label(img_addr) # get the unique ID
            train_images.append(img) # append the images
            img_lbl = np.array([img_lbl]) 
            train_labels.append(img_lbl) # append the labels
        x_train=np.array(train_images, dtype=np.float32)/255 #normalize the images
        noise_factor = 0.5
        #create noise in the images and save it as x_train_noisy
        x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
        x_train_noisy = np.clip(x_train_noisy, 0., 1.)
        
        # train the autoencoder with x = x_train_noisy and y = x_train
        model.fit(x_train_noisy, x_train, batch_size=16, epochs=epochs)
        
    return model

In [16]:
epochs=50
autoencoder_train=train_model(model,batches_list,batch_size,train_sample,epochs)

Epoch 1/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0553
Epoch 2/50
4/4 [==============================] - 1s 285ms/step - loss: 0.0553
Epoch 3/50
4/4 [==============================] - 1s 290ms/step - loss: 0.0552
Epoch 4/50
4/4 [==============================] - 1s 289ms/step - loss: 0.0552
Epoch 5/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0551
Epoch 6/50
4/4 [==============================] - 1s 294ms/step - loss: 0.0551
Epoch 7/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0550
Epoch 8/50
4/4 [==============================] - 1s 308ms/step - loss: 0.0550
Epoch 9/50
4/4 [==============================] - 1s 284ms/step - loss: 0.0550
Epoch 10/50
4/4 [==============================] - 1s 294ms/step - loss: 0.0549
Epoch 11/50
4/4 [==============================] - 1s 292ms/step - loss: 0.0549
Epoch 12/50
4/4 [==============================] - 1s 285ms/step - loss: 0.0548
Epoch 13/50
4/4 [==============================] 

4/4 [==============================] - 1s 289ms/step - loss: 0.0511
Epoch 6/50
4/4 [==============================] - 1s 289ms/step - loss: 0.0511
Epoch 7/50
4/4 [==============================] - 1s 289ms/step - loss: 0.0511
Epoch 8/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0511
Epoch 9/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0511
Epoch 10/50
4/4 [==============================] - 1s 289ms/step - loss: 0.0511
Epoch 11/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0511
Epoch 12/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0510
Epoch 13/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0510
Epoch 14/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0510
Epoch 15/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0510
Epoch 16/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0510
Epoch 17/50
4/4 [==============================] - 1s 28

4/4 [==============================] - 1s 288ms/step - loss: 0.0510
Epoch 10/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0510
Epoch 11/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0510
Epoch 12/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0510
Epoch 13/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0510
Epoch 14/50
4/4 [==============================] - 1s 289ms/step - loss: 0.0510
Epoch 15/50
4/4 [==============================] - 1s 285ms/step - loss: 0.0510
Epoch 16/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0510
Epoch 17/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0510
Epoch 18/50
4/4 [==============================] - 1s 290ms/step - loss: 0.0510
Epoch 19/50
4/4 [==============================] - 1s 285ms/step - loss: 0.0510
Epoch 20/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0510
Epoch 21/50
4/4 [==============================] - 1

4/4 [==============================] - 1s 285ms/step - loss: 0.0521
Epoch 14/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0521
Epoch 15/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0521
Epoch 16/50
4/4 [==============================] - 1s 290ms/step - loss: 0.0520
Epoch 17/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0520
Epoch 18/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0520
Epoch 19/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0520
Epoch 20/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0519
Epoch 21/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0519
Epoch 22/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0519
Epoch 23/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0519
Epoch 24/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0518
Epoch 25/50
4/4 [==============================] - 1

4/4 [==============================] - 1s 290ms/step - loss: 0.0388
Epoch 18/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0388
Epoch 19/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0387
Epoch 20/50
4/4 [==============================] - 1s 290ms/step - loss: 0.0387
Epoch 21/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0386
Epoch 22/50
4/4 [==============================] - 1s 289ms/step - loss: 0.0385
Epoch 23/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0385
Epoch 24/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0384
Epoch 25/50
4/4 [==============================] - 1s 289ms/step - loss: 0.0384
Epoch 26/50
4/4 [==============================] - 1s 289ms/step - loss: 0.0383
Epoch 27/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0383
Epoch 28/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0382
Epoch 29/50
4/4 [==============================] - 1

...

4/4 [==============================] - 1s 290ms/step - loss: 0.0188
Epoch 42/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0188
Epoch 43/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0188
Epoch 44/50
4/4 [==============================] - 1s 291ms/step - loss: 0.0188
Epoch 45/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0188
Epoch 46/50
4/4 [==============================] - 1s 291ms/step - loss: 0.0188
Epoch 47/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0188
Epoch 48/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0188
Epoch 49/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0188
Epoch 50/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0188
Epoch 1/50
4/4 [==============================] - 1s 285ms/step - loss: 0.0198
Epoch 2/50
4/4 [==============================] - 1s 290ms/step - loss: 0.0198
Epoch 3/50
4/4 [==============================] - 1s 2

4/4 [==============================] - 1s 284ms/step - loss: 0.0162
Epoch 46/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0162
Epoch 47/50
4/4 [==============================] - 1s 285ms/step - loss: 0.0162
Epoch 48/50
4/4 [==============================] - 1s 289ms/step - loss: 0.0162
Epoch 49/50
4/4 [==============================] - 1s 284ms/step - loss: 0.0162
Epoch 50/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0162
Epoch 1/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0155
Epoch 2/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0155
Epoch 3/50
4/4 [==============================] - 1s 292ms/step - loss: 0.0155
Epoch 4/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0155
Epoch 5/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0155
Epoch 6/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0155
Epoch 7/50
4/4 [==============================] - 1s 286ms

4/4 [==============================] - 1s 289ms/step - loss: 0.0171
Epoch 50/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0171
Epoch 1/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0182
Epoch 2/50
4/4 [==============================] - 1s 291ms/step - loss: 0.0182
Epoch 3/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0182
Epoch 4/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0181
Epoch 5/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0182
Epoch 6/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0181
Epoch 7/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0182
Epoch 8/50
4/4 [==============================] - 1s 289ms/step - loss: 0.0181
Epoch 9/50
4/4 [==============================] - 1s 290ms/step - loss: 0.0181
Epoch 10/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0181
Epoch 11/50
4/4 [==============================] - 1s 288ms/s

4/4 [==============================] - 1s 286ms/step - loss: 0.0168
Epoch 4/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0168
Epoch 5/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0168
Epoch 6/50
4/4 [==============================] - 1s 286ms/step - loss: 0.0168
Epoch 7/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0168
Epoch 8/50
4/4 [==============================] - 1s 284ms/step - loss: 0.0168
Epoch 9/50
4/4 [==============================] - 1s 284ms/step - loss: 0.0168
Epoch 10/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0168
Epoch 11/50
4/4 [==============================] - 1s 284ms/step - loss: 0.0168
Epoch 12/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0168
Epoch 13/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0168
Epoch 14/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0168
Epoch 15/50
4/4 [==============================] - 1s 287m

4/4 [==============================] - 1s 288ms/step - loss: 0.0171
Epoch 8/50
4/4 [==============================] - 1s 290ms/step - loss: 0.0171
Epoch 9/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0171
Epoch 10/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0170
Epoch 11/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0171
Epoch 12/50
4/4 [==============================] - 1s 290ms/step - loss: 0.0171
Epoch 13/50
4/4 [==============================] - 1s 289ms/step - loss: 0.0171
Epoch 14/50
4/4 [==============================] - 1s 288ms/step - loss: 0.0170
Epoch 15/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0170
Epoch 16/50
4/4 [==============================] - 1s 287ms/step - loss: 0.0171
Epoch 17/50
4/4 [==============================] - 1s 290ms/step - loss: 0.0170
Epoch 18/50
4/4 [==============================] - 1s 292ms/step - loss: 0.0170
Epoch 19/50
4/4 [==============================] - 1s 

In [17]:
model.save('autoencoder.h5')